In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

In [4]:
load_dotenv()

index_name="test-index-tom"

model = AzureChatOpenAI(
    openai_api_version=str(os.getenv("AZURE_OPENAI_API_VERSION")),
    azure_deployment=str(os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")),
)

AZURE_SEARCH_KEY = str(os.getenv("AZURE_SEARCH_KEY"))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="orisai-text-embedding-3-large-development",
)

retriever = AzureAISearchRetriever(
    content_key="content", index_name=index_name, service_name="orisai-search-development", api_key=AZURE_SEARCH_KEY, top_k=1
)

In [10]:
# Prompt template
template = """Answer the question based on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = model

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

async for chunk in rag_chain.astream("Wat is de woonfiscaliteit in brussel?"):
    print(chunk, end="", flush=True)

Het Brusselse Gewest heft de hoogste registratierechten in vergelijking met andere regio's in België. In Brussel bedragen de registratierechten 12,5%. Dit wordt echter gecompenseerd door de zogenaamde abattementen (som waarop geen belasting wordt betaald). Op dit moment hoeft er op de eerste 175.000 euro van de woningprijs geen belasting te worden betaald. Vanaf 2023 wordt dit bedrag verhoogd naar de eerste 200.000 euro.

In [15]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
You can use the following pieces of retrieved context to answer the question. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
conversational_rag_chain.invoke(
    {"input": "Wat is woningfiscaliteit?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Woningfiscaliteit verwijst naar de belastingen en heffingen die gerelateerd zijn aan het bezit, de aankoop, verkoop of verhuur van een woning. Dit kan onder meer registratierechten, onroerende voorheffing, en belasting op huurinkomsten omvatten. In het gegeven context, verwijst het naar de registratierechten die betaald worden bij de aankoop van een woning, die in Brussel hoger zijn dan in Vlaanderen.'

In [2]:
conversational_rag_chain.invoke(
    {"input": "hoe zit dit in brussel?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'In 2021 verhuisden een recordaantal van 24.000 Brusselaars naar Vlaanderen. Deze trend wordt toegeschreven aan de hoge prijzen in Brussel, het effect van de coronapandemie en veranderde woonwensen, en fiscale factoren. Daarnaast is de Brusselse huurmarkt ook onder druk door krapte, wat bijdraagt aan de stadsvlucht.'